In [1]:
!pip install huggingface-hub
!pip install datasets
!pip install trl
!pip install peft
!pip install transformers
!pip install bitsandbytes
!pip install accelerate
!pip install flash-attn

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
# Import the 'login' function from the huggingface_hub library to authenticate and access your Hugging Face account.
from huggingface_hub import login
# Call the login function to authenticate. You'll need to enter your credentials or token.
login(token='')

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/jovyan/.cache/huggingface/token
Login successful


In [3]:
import torch
import os
import sys
import json
import pandas as pd
from IPython.display import display, HTML
from datetime import datetime
import datasets
from datasets import load_dataset

import torch
import trl
from trl import setup_chat_format,SFTTrainer

import peft
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model, AutoPeftModelForCausalLM
import transformers
from transformers import (
    LlavaForConditionalGeneration,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline
)

from random import randint
print("Torch version:", torch.__version__)
print("Datasets version:", datasets.__version__)
print("TRL (Transformers Reinforcement Learning) version:", trl.__version__)
print("PEFT (Parameter Efficient Fine-Tuning) version:", peft.__version__)
print("Transformers version:", transformers.__version__)

2024-06-18 19:46:53.446810: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-18 19:46:53.486477: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-18 19:46:54.127071: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Torch version: 2.2.2+cu121
Datasets version: 2.20.0
TRL (Transformers Reinforcement Learning) version: 0.9.4
PEFT (Parameter Efficient Fine-Tuning) version: 0.11.1
Transformers version: 4.41.2


In [4]:
# Define the identifier for the model to be loaded, specifically "meta-llama/Meta-Llama-3-8B".
model_id = "llava-hf/llava-v1.6-mistral-7b-hf" 

# Configure the model for low-precision (4-bit) quantization to reduce memory usage and potentially increase inference speed.
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Enable loading the model in 4-bit precision
    bnb_4bit_use_double_quant=True,  # Use double quantization for better precision handling
    bnb_4bit_quant_type="nf4",  # Set the quantization type to 'nf4', a specific 4-bit format
    bnb_4bit_compute_dtype=torch.bfloat16  # Use bfloat16 as the datatype for computation to balance performance and accuracy
)

# Load the pre-trained causal language model from Hugging Face's model hub.
# The model is automatically distributed across available GPUs if possible using `device_map="auto"`.
model = LlavaForConditionalGeneration.from_pretrained(
    model_id,
    device_map="auto",  # Automatically distribute the model across available GPUs
    attn_implementation="flash_attention_2",  # Use an optimized attention mechanism for better performance
    torch_dtype=torch.bfloat16,  # Use bfloat16 as the default tensor data type for all model parameters
    quantization_config=bnb_config  # Apply the defined quantization configuration
)

config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

You are using a model of type llava_next to instantiate a model of type llava. This is not supported for all configurations of models and can yield errors.
/opt/conda/lib/python3.11/site-packages/transformers/models/llava/configuration_llava.py:100: FutureWarning: The `vocab_size` argument is deprecated and will be removed in v4.42, since it can be inferred from the `text_config`. Passing this argument has no effect
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/70.2k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/380M [00:00<?, ?B/s]

You are attempting to use Flash Attention 2.0 without specifying a torch dtype. This might lead to unexpected behaviour
The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [5]:
model

LlavaForConditionalGeneration(
  (vision_tower): CLIPVisionModel(
    (vision_model): CLIPVisionTransformer(
      (embeddings): CLIPVisionEmbeddings(
        (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
        (position_embedding): Embedding(577, 1024)
      )
      (pre_layrnorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (encoder): CLIPEncoder(
        (layers): ModuleList(
          (0-23): 24 x CLIPEncoderLayer(
            (self_attn): CLIPAttention(
              (k_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
            )
            (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (mlp): CLIPMLP(
              (activ

In [6]:
model.save_pretrained("model", from_pt=True) 